In [ ]:
import pandas as pd

# Definition du chemin vers le fichier source
path_in = 'Data/concat.csv'

# Choix entre 'star' et 'director'
pers_type = 'director' 

# Charger le DataFrame
df = pd.read_csv(path_in)

# Fonction pour supprimer les doublons tout en conservant l'ordre
def supprimer_doublons(valeurs):
    if pd.isna(valeurs):
        return ''
    valeurs_liste = valeurs.split(',')
    valeurs_uniques = list(dict.fromkeys(valeurs_liste))
    return ','.join(valeurs_uniques)

# Appliquer la fonction pour supprimer les doublons
df[pers_type] = df[pers_type].apply(supprimer_doublons)
df[f'{pers_type}_id'] = df[f'{pers_type}_id'].apply(supprimer_doublons)

# Split les chaînes de caractères dans les colonnes `star` et `star_id`
df[pers_type] = df[pers_type].str.split(', ')
df[f'{pers_type}_id'] = df[f'{pers_type}_id'].str.split(',')

# S'assurer que chaque élément de la liste `star` est associé à son `star_id`
df_final = df.apply(lambda x: pd.Series(zip(x[pers_type], x[f'{pers_type}_id'])), axis=1).stack().reset_index(level=1, drop=True).apply(pd.Series)

# Renommer les colonnes du DataFrame final
df_final.columns = [pers_type, f'{pers_type}_id']

df_clean = df_final.dropna(how='all')

df_clean[f'{pers_type}_id'] = df_clean[f'{pers_type}_id'].str.replace('/name/', '')
df_clean[f'{pers_type}_id'] = df_clean[f'{pers_type}_id'].str.replace('/', '')

# Compter le nombre de lignes avant la suppression des doublons
nombre_lignes_avant = len(df_clean)

# Suppression des lignes dupliquées
df_clean = df_clean.drop_duplicates()

# Compter le nombre de lignes après la suppression des doublons
nombre_lignes_apres = len(df_clean)

# Calculer le nombre de lignes supprimées
nombre_lignes_supprimees = nombre_lignes_avant - nombre_lignes_apres

# Sauvegarder le résultat
df_clean.to_csv(f'Data/nom_imdbID_{pers_type}.csv', index=False)